# Lista Pandas - Desempenho dos Alunos GDF

O dataset analisado aqui descreve o desempenho dos alunos da rede pública do Distrito Federal. Nele encontramos as notas bimestrais de todas as matérias oferecidas a cada aluno do GDF, juntamente com informações básicas sobre a escola, o curso e a série do aluno. A fonte dos dados é o portal de dados abertos do GDF.

Utilize o dataset para responder as questões abaixo. Coloque sua solução somente dentro do espaço delimitado para a resposta e não altere nenhum outro código disponibilizado. As bibliotecas para a resolução de todos os problemas já foram importadas na célula abaixo.

Obs.: Caso alguma delas não esteja instalada no seu ambiente execute o processo de instalção através de !pip install <nome_da_biblioteca>



In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import requests as r
from zipfile import ZipFile
import io

import warnings
warnings.filterwarnings('ignore')

In [2]:
def get_data(rows=None,download=True) -> pd.core.frame.DataFrame:
    
    """ get_data(rows: int=None, download: bool=True) -> pandas.core.frame.DataFrame:
        
        - Retorna os dados para serem utilizados na resolução da lista. 
        - Utilize o parâmetro 'download' caso você já possua os dados na sua pasta de trabalho.
        
        
        ** Parâmetros **
        ________________
        
        rows : quantidade de linhas a serem lidas no dataset, padrão é None e representa todo o dataset.
        download : define se a função executará o processo de download e extração dos dados, padrão True.
        
        ** Exemplos **
        ________________
        
        >> df = get_data(rows=100)
        >> df.shape
        >> (100,23)
        
        
        >> df = get_data(rows=578)
        >> df.shape
        >> (578,23)
        
    """
    
    if download == True:
            
        file_zipped = r.get('http://dados.df.gov.br/dataset/b8436049-44e7-4224-95b4-224718a4b166/resource/3e654a9d-0647-4e39-930e-7cd07faec888/download/dados-abertos---desempenho-escolar20180515160111.zip')  
        z = ZipFile(io.BytesIO(file_zipped.content))
        z.extractall('.')
    
    return pd.read_csv('dados abertos - desempenho escolar_20180515_160111.csv',sep=';',encoding='latin-1',nrows=rows)

In [3]:
df = get_data()

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12779084 entries, 0 to 12779083
Data columns (total 23 columns):
 #   Column                            Dtype  
---  ------                            -----  
 0   cod_coordenacao_regional          int64  
 1   coordenacao_regional              object 
 2   cod_escola                        int64  
 3   escola                            object 
 4   cod_curso                         int64  
 5   curso                             object 
 6   cod_serie                         int64  
 7   serie                             object 
 8   cod_turno                         int64  
 9   turno                             object 
 10  cod_turma                         float64
 11  turma                             object 
 12  cod_aluno                         int64  
 13  bimestre                          object 
 14  cod_disciplina                    int64  
 15  disciplina                        object 
 16  nota_bimestral                    

In [ ]:
df.head()

,cod_coordenacao_regional,coordenacao_regional,cod_escola,escola,cod_curso,curso,cod_serie,serie,cod_turno,turno,cod_turma,turma,cod_aluno,bimestre,cod_disciplina,disciplina,nota_bimestral,aulas_bimestre,faltas_bimestre,faltas_justificadas_bimestre,aulas_globalizadas,faltas_globalizadas,faltas_justificadas_globalizadas
0,3,CRE - Plano Piloto,1,CENTRO DE ENSINO FUNDAMENTAL POLIVALENTE,5,Ensino Fundamental de 9 Anos,8,6º Ano,5,Diurno,100329.0,6º ANO B,-8376490126953895597,1,22,Projeto Interdisciplinar I,9.0,10.0,0.0,0.0,NaN,NaN,NaN
1,3,CRE - Plano Piloto,1,CENTRO DE ENSINO FUNDAMENTAL POLIVALENTE,5,Ensino Fundamental de 9 Anos,8,6º Ano,5,Diurno,100329.0,6º ANO B,-8376490126953895597,2,22,Projeto Interdisciplinar I,8.3,10.0,0.0,0.0,NaN,NaN,NaN
2,3,CRE - Plano Piloto,1,CENTRO DE ENSINO FUNDAMENTAL POLIVALENTE,5,Ensino Fundamental de 9 Anos,8,6º Ano,5,Diurno,100329.0,6º ANO B,-8376490126953895597,3,22,Projeto Interdisciplinar I,6.5,10.0,0.0,0.0,NaN,NaN,NaN
3,3,CRE - Plano Piloto,1,CENTRO DE ENSINO FUNDAMENTAL POLIVALENTE,5,Ensino Fundamental de 9 Anos,8,6º Ano,5,Diurno,100329.0,6º ANO B,-8376490126953895597,4,22,Projeto Interdisciplinar I,9.5,9.0,0.0,0.0,NaN,NaN,NaN
4,3,CRE - Plano Piloto,1,CENTRO DE ENSINO FUNDAMENTAL POLIVALENTE,5,Ensino Fundamental de 9 Anos,8,6º Ano,5,Diurno,100329.0,6º ANO B,-8376490126953895597,resultado final,22,Projeto Interdisciplinar I,8.5,NaN,NaN,NaN,NaN,NaN,NaN


## Ex 01.
 Quantos alunos diferentes estão matriculados no curso "Ensino Médio"?

In [ ]:
qtd_alunos_em = df.loc[df['curso'] == 'Ensino Médio']['cod_aluno'].nunique()
print(qtd_alunos_em)

81712


## Ex 02. 
Levando em consideração somente os alunos do curso "Ensino Médio". Descreva a porcentagem de alunos que foram aprovados e reprovados (aluno aprovado é aquele que tirou 5 ou mais como nota final em todas as matérias cursadas) por série.

In [ ]:
# verifica as séries existenstes para Ensino Médio no dataset
df[df.curso == 'Ensino Médio'].serie.unique()

array(['1ª Série', '2ª Série', '3ª Série'], dtype=object)

In [ ]:
# verifica se existem notas acima de 10 para avaliar a necessidade de colocar na mesma escala
df.loc[(df['curso'] == 'Ensino Médio') & (df['bimestre'] == 'resultado final') & (df['nota_bimestral'] > 10)]['nota_bimestral'].value_counts()


In [ ]:
# quantidade de disciplinas diferentes
disciplinas_em = df.loc[(df['curso'] == 'Ensino Médio') & (df['bimestre'] == 'resultado final')]['disciplina'].unique()
print(format('Disciplinas do EM', '*^82'))
print(disciplinas_em, end='\n\n')
print(format('Qtd. Disciplinas do EM', '*^82'))
print(len(disciplinas_em))

********************************Disciplinas do EM*********************************
['Sociologia' 'Língua Estrangeira - Espanhol' 'Ensino Religioso'
 'Língua Estrangeira - Inglês' 'Matemática' 'Língua Portuguesa' 'História'
 'Educação Física' 'Projeto Interdisciplinar II' 'Geografia'
 'Projeto Interdisciplinar I' 'Projeto Interdisciplinar III' 'Química'
 'Biologia' 'Física' 'Filosofia' 'Arte']

******************************Qtd. Disciplinas do EM******************************
17


In [ ]:
# alunos do ensimo médio por disciplina
(
df.loc[(df['curso'] == 'Ensino Médio') & (df['bimestre'] == 'resultado final')]
.groupby('disciplina')
.agg({'cod_aluno': 'count'})
)

,cod_aluno
disciplina,
Arte,77087
Biologia,77098
Educação Física,77114
Ensino Religioso,28190
Filosofia,76965
Física,77116
Geografia,77099
História,77103
Língua Estrangeira - Espanhol,77065


- Analisando a saida da celula anterior (qtd. alunos por disciplina), resovi retirar da conta as disciplinas de Projeto interdisciplinar I, II e III, e Ensino religioso para considerar o percentual de aprovados, já que essas disciplinas tem 70k ou menos alunos enquanto as demais possuem > 77k.

Então ao invés de considerar 17 disciplinas, irei considerar 17 - 4 = 13

In [ ]:
# função que soma as notas bimestrais que são >= 5
def qtd_materias_aprovado(nota_series):
  
  return (nota_series.apply(lambda x: round(x/10, 2) if x > 10 else x) >= 5).sum()

In [ ]:
for serie in df[df['curso'] == 'Ensino Médio'].serie.unique():

  qtd_alunos_serie = df.loc[(df['curso'] == 'Ensino Médio') &(df['serie'] == serie)]['cod_aluno'].nunique()
  qtd_aprovados_serie = ((
      df.loc[(df['curso'] == 'Ensino Médio') & (df['bimestre'] == 'resultado final')&(df['serie'] == serie)]
      .groupby(['cod_aluno'])
      .agg({'nota_bimestral': qtd_materias_aprovado})
    ) >= 13).sum()['nota_bimestral']

  print(format(f'Alunos da {serie}', '*^82'))
  print(f' -> porcentagem de alunos aprovados: {round((qtd_aprovados_serie/qtd_alunos_serie)*100, 2 )}%')
  print(f' -> porcentagem de alunos reprovados: {round((1 - (qtd_aprovados_serie/qtd_alunos_serie))*100, 2 )}%', end='\n\n')


********************************Alunos da 1ª Série********************************
 -> porcentagem de alunos aprovados: 75.31%
 -> porcentagem de alunos reprovados: 24.69%

********************************Alunos da 2ª Série********************************
 -> porcentagem de alunos aprovados: 81.78%
 -> porcentagem de alunos reprovados: 18.22%

********************************Alunos da 3ª Série********************************
 -> porcentagem de alunos aprovados: 88.19%
 -> porcentagem de alunos reprovados: 11.81%



In [ ]:
----------------------------------------------
#  Outra forma de implementar
----------------------------------------------

# def corrigir_nota(value):
#     if value > 10:
#         return round(value / 10, 2)
#     else:
#         return value


# (((
# df[(df.curso == 'Ensino Médio')&(df.bimestre == 'resultado final')&(df.serie == '2ª Série')]
# .loc[:,['cod_aluno','serie','disciplina','nota_bimestral']]
# .assign(nota_bimestral = lambda x: x.nota_bimestral.apply(corrigir_nota))
# .pivot_table(index='cod_aluno',columns='disciplina',values='nota_bimestral',aggfunc='max')
# .fillna(0)
# )  >= 5).sum(axis=1) >= 13).sum()

17994

##Ex 03. 
As coordenações regionais são instituições que cuidam das escolas públicas de uma determinada região. Busque dados populacionais de cada coordenação regional e demonstre se há alguma correlação entra a quantidade de alunos de cada coordenação e a população daquela cidade satélite.

In [ ]:
df_popdf = pd.read_html('https://pt.wikipedia.org/wiki/Lista_de_regi%C3%B5es_administrativas_do_Distrito_Federal_por_popula%C3%A7%C3%A3o')[0]

display(df_popdf.head())
display(df_popdf.info())

,Posição,Região administrativa,População
0,1,Ceilândia,489 351
1,2,Samambaia,254 439
2,3,Taguatinga,222 598
3,4,Plano Piloto,220 393
4,5,Planaltina,189 421


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 3 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Posição                31 non-null     int64 
 1   Região administrativa  31 non-null     object
 2   População              31 non-null     object
dtypes: int64(1), object(2)
memory usage: 872.0+ bytes


None

In [ ]:
df_popdf['População'] = df_popdf['População'].apply(lambda x: int(''.join(x.split())))

In [ ]:
display(df_popdf['População'].head())
display(df_popdf['População'].dtype)

0    489351
1    254439
2    222598
3    220393
4    189421
Name: População, dtype: int64

dtype('int64')

In [ ]:
correlacao = (
df[['coordenacao_regional', 'cod_aluno']]
.drop_duplicates(subset='cod_aluno')
.groupby('coordenacao_regional')
.count()
.reset_index()
.rename(columns={'cod_aluno': 'qtd_alunos'})
.assign(nome_coord_tratado = lambda x: x['coordenacao_regional'].apply(lambda y: y[6:]))
.merge(df_popdf[['Região administrativa', 'População']], left_on='nome_coord_tratado', right_on='Região administrativa')
.loc[:, ['qtd_alunos', 'População']]
.corr()
)
  # print(format(f'Correleção entre a qtd. de alunos e a população de {coord_reg}','*^82'))
print(correlacao, end='\n\n')

            qtd_alunos  População
qtd_alunos    1.000000   0.937452
População     0.937452   1.000000



> Resposta 03: Percebe-se uma forte correlação entre quantidade de alunos das CRE's com a população da respectiva cidade satélite

##Ex 04. 
Levando em consideração somente os alunos do curso "Ensino Médio". Demonstre a porcentagem de alunos que obtiveram notas finais maiores ou iguais a 8 em Química, Física e Biologia, por coordenação Regional.

In [ ]:
df['coordenacao_regional'].unique()

array(['CRE - Plano Piloto', 'CRE - Brazlândia', 'CRE - Ceilândia',
       'CRE - Gama', 'CRE - Guará', 'CRE - Núcleo Bandeirante',
       'CRE - Planaltina', 'CRE - Sobradinho', 'CRE - Taguatinga',
       'CRE - Recanto das Emas', 'CRE - Samambaia', 'CRE - Paranoá',
       'CRE - Santa Maria', 'CRE - São Sebastião'], dtype=object)

In [ ]:
def corrigir_nota(value):
    if value > 10:
        return round(value / 10, 2)
    else:
        return value
  

lista_disciplinas = ['Física', 'Química', 'Biologia']
for coord_reg in df['coordenacao_regional'].unique():

  qtd_alunos_regional = df.query('curso == "Ensino Médio" and bimestre == "resultado final" and coordenacao_regional == @coord_reg')['cod_aluno'].nunique()

  qtd_aprovados_reg = (((
  df.query('curso == "Ensino Médio" and bimestre == "resultado final" and disciplina in(@lista_disciplinas) and coordenacao_regional == @coord_reg')
  .loc[:,['cod_aluno','disciplina','nota_bimestral']]
  .assign(nota_bimestral = lambda x: x.nota_bimestral.apply(corrigir_nota))
  .pivot_table(index='cod_aluno',columns='disciplina',values='nota_bimestral',aggfunc='max')
  .fillna(0)
  ) >= 8).sum(axis=1) == 3).sum()

  print(format(f'{coord_reg}', '*^82'))
  print(f' -> porcentagem de alunos que obtiveram notas finais maiores ou iguais a 8 em Química, Física e Biologia: {round((qtd_aprovados_reg/qtd_alunos_regional)*100, 2 )}%')
  

********************************CRE - Plano Piloto********************************
 -> porcentagem de alunos que obtiveram notas finais maiores ou iguais a 8 em Química, Física e Biologia: 9.49%
*********************************CRE - Brazlândia*********************************
 -> porcentagem de alunos que obtiveram notas finais maiores ou iguais a 8 em Química, Física e Biologia: 11.27%
*********************************CRE - Ceilândia**********************************
 -> porcentagem de alunos que obtiveram notas finais maiores ou iguais a 8 em Química, Física e Biologia: 14.98%
************************************CRE - Gama************************************
 -> porcentagem de alunos que obtiveram notas finais maiores ou iguais a 8 em Química, Física e Biologia: 9.71%
***********************************CRE - Guará************************************
 -> porcentagem de alunos que obtiveram notas finais maiores ou iguais a 8 em Química, Física e Biologia: 16.85%
**********************

In [ ]:
#--------------------------------------------------
# Outra forma de fazer
#--------------------------------------------------

## função que soma as notas bimestrais que são >= 8
# def qtd_materias_aprovado(nota_series):
  
#   return (nota_series.apply(lambda x: round(x/10, 2) if x > 10 else x) >= 8).sum()

# ((
#   df.query('curso == "Ensino Médio" and bimestre == "resultado final" and disciplina in(@lista_disciplinas) and coordenacao_regional == "CRE - Plano Piloto"')
#   .groupby(['cod_aluno'])
#   .agg({'nota_bimestral': qtd_materias_aprovado})
# ) == 3).sum()

##Ex 05. 
Qual coordenação regional tem a maior quantidade de alunos matriculados no turno diurno do curso "Ensino Médio".Considere somente as coordenações regionais que possuem um número acima da média de aprovados.

In [ ]:
# Total de alunos aprovados do Ensino médio que estudam no turno Diurno, por Coordenação Regional
s_aprovados_em_diurno_pcoord = (
df[['coordenacao_regional', 'cod_aluno', 'curso', 'bimestre', 'turno', 'nota_bimestral']]
.query('curso == "Ensino Médio" and bimestre == "resultado final" and turno == "Diurno"')
.groupby(['coordenacao_regional','cod_aluno'])
.agg(qtd_mat_aprovado=('nota_bimestral', qtd_materias_aprovado))
.assign(aprovado_final = lambda x: x['qtd_mat_aprovado'] >= 13)
.groupby('coordenacao_regional')['aprovado_final'].sum()
)
display(s_aprovados_em_diurno_pcoord)

coordenacao_regional
CRE - Brazlândia             1919
CRE - Ceilândia             10382
CRE - Gama                   4405
CRE - Guará                  2331
CRE - Núcleo Bandeirante     2803
CRE - Paranoá                2565
CRE - Planaltina             4925
CRE - Plano Piloto           6931
CRE - Recanto das Emas       3941
CRE - Samambaia              3569
CRE - Santa Maria            3236
CRE - Sobradinho             3438
CRE - São Sebastião          2772
CRE - Taguatinga             7412
Name: aprovado_final, dtype: int64

In [ ]:
# validando os resultados
s_aprovados_em_diurno_pcoord.sum()

60629

In [ ]:
# Total de alunos aprovados do Ensino Médio Diurno 
((
      df.loc[(df['curso'] == 'Ensino Médio') & (df['bimestre'] == 'resultado final') & (df['turno'] == 'Diurno')]
      .groupby(['cod_aluno'])
      .agg({'nota_bimestral': qtd_materias_aprovado})
    ) >= 13).sum()

nota_bimestral    60629
dtype: int64

In [ ]:
# Média dos alunos aprovados no Ensino Médio que estudam no turno Diurno
media_aprovados_em_diurno = s_aprovados_em_diurno_pcoord.mean()
display(media_aprovados_em_diurno)

4330.642857142857

In [ ]:
#  coordenação regional que tem a maior quantidade de alunos matriculados no turno diurno do curso "Ensino Médio"
s_aprovados_em_diurno_pcoord.where(s_aprovados_em_diurno_pcoord > media_aprovados_em_diurno).nlargest(1)

coordenacao_regional
CRE - Ceilândia    10382.0
Name: aprovado_final, dtype: float64

##Ex 06. 
Considerando somente alunos do curso "Ensino Médio". Os alunos do turno diurno perfomam melhor que os alunos do turno noturno nas matérias de química, física e biologia. (Verdadeiro ou Falso, demonstre com dados).



In [15]:
def corrigir_nota(value):
    if value > 10:
        return round(value / 10, 2)
    else:
        return value
  

lista_disciplinas = ['Física', 'Química', 'Biologia']
dict_medias = {}
for turno in df['turno'].unique():

  media_p_turno = ((
  df.query('curso == "Ensino Médio" and bimestre == "resultado final" and disciplina in(@lista_disciplinas) and turno == @turno')
  .loc[:,['cod_aluno','disciplina','nota_bimestral']]
  .assign(nota_bimestral = lambda x: x['nota_bimestral'].apply(corrigir_nota))
  .pivot_table(index='cod_aluno',columns='disciplina',values='nota_bimestral',aggfunc='max')
  .fillna(0)
  ).mean(axis=1)).mean()

  print(format(f'Turno: {turno}', '*^82'))
  print(f' -> A nota média dos alunos do Ensino Médio - {turno} nas disciplinas Química, Física e Biologia é de: {round(media_p_turno, 2 )}')
  dict_medias[turno] = media_p_turno


sorted_dict = sorted(dict_medias.items(), key=lambda x: x[1])
print(f'\n >> Logo, os alunos do turno {sorted_dict[1][0]} perfomam melhor que os alunos do turno {sorted_dict[0][0]} nas disciplinas Química, Física e Biologia.')

**********************************Turno: Diurno***********************************
 -> A nota média dos alunos do Ensino Médio - Diurno nas disciplinas Química, Física e Biologia é de: 5.91
**********************************Turno: Noturno**********************************
 -> A nota média dos alunos do Ensino Médio - Noturno nas disciplinas Química, Física e Biologia é de: 7.2

 >> Logo, os alunos do turno Noturno perfomam melhor que os alunos do turno Diurno nas disciplinas Química, Física e Biologia.
